# Daiichi - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("ToV_2018_CH.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df)

## Format Table

In [18]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'address', 'plz', 'location', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']

#Shift
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Remove empty rows
df_export = df_export.dropna(subset=['plz'], how='all')
df_export = df_export.reset_index(drop=True)

#Drop by name
df_export.drop(df_export[df_export['name'].str.contains('Gesamtbetrag', na=False, regex=False)].index, inplace=True)
df_export.drop(df_export[df_export['name'].str.contains('Gesamtzahl der', na=False, regex=False)].index, inplace=True)
df_export.drop(df_export[df_export['name'].str.contains('Prozentualer Anteil', na=False, regex=False)].index, inplace=True)

#drop before RnD
df_export = df_export.reset_index(drop=True)
rnd = df_export[df_export['name'].str.contains('Forschung & Entwicklung', na=False, regex=False)].index[0]
df_export = df_export[0: rnd]

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Add Type
df_export = set_type_by_alphabetical_order(df_export)

#Revert Name
df_export = revert_name(df_export, ' ')

#Cleanup Numbers and sum
df_export = remove_in_numbers(df_export, 'Fr.')
df_export = remove_in_numbers(df_export, 'n.a.')
df_export = remove_in_numbers(df_export, ' ')
df_export = remove_in_numbers(df_export, '’')
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add empty fields
df_export['country'] = 'CH'
df_export = add_uci(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'daiichi')

Duplicates found. Please check for duplicates: df_export[df_export.duplicated()]
saved


In [116]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)